## Requirements Relationships Chord Diagram


In [34]:
import re
from itertools import combinations
import psycopg2
import numpy as np
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from IPython import display
from bs4 import BeautifulSoup as bs

from nbstyler import DATA_STYLE as style

plotly.offline.init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline

%matplotlib notebook
%matplotlib inline

### Data Preparation

In [2]:
data_querystr = """SELECT * FROM v_full_data_offers_history"""
conn = psycopg2.connect('dbname=jobsbg')
data_df = pd.read_sql_query(data_querystr, conn, index_col='subm_date')
conn.close()

In [3]:
data_df.head(1)

,subm_type,job_id,company_id,norm_salary,job_title,company_name,text_salary,job_contents
subm_date,,,,,,,,
2017-09-27,submission,3994437,124912,NaN,Data Analyst,ПрайсуотърхаусКупърс Одит ООД,None,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 ..."


### Relationship Categories Definition

In order to build the necessary data we first need to define the scope of the relationships we aim to observe. The search strings we will compare should be of the same semantic category to provide meaningful insight.

To explore the data and prepare proper definitions we took a deep dive in the data offers' text contents. We used the `nltk` library for text processing to produce frequency distributions of most common words, bigrams and trigrams. Then those distributions were used to explore different possible relationship categories. See the details in the [Requirements Deep Dive Notebook](./Data_Offers_Requirements_Deep_Dive.ipynb).

For the first chord diagram the chosen view is a breakdown by technology. Other possible relationships to explore are skill requirements breakdown, perks & benefits breakdown, job title breakdown, etc.

### Data Jobs Technology Requirement Relationships

A preliminary list of terms that describe most often sought categories is presented below. We are going to use it to construct regex filters for selected key technologies.

In [4]:
tech_terms_candidates = [
    'excel', 'tableau', 'access', 'qlik', 'hadoop', 'informatica', 'vmware', 'ssis', 'vba', 'python','powerpoint', 'mysql',
    'spark', 'microstrategy', 'deluge', 'ssrs', ('sql', 'server'), ('power', 'bi'), ('ms', 'office'), ('microsoft', 'office'), ]

In [5]:
tech_terms_filters = [
    r'excel', r'tableau', r'qlik', r'hadoop', r'ss[ir]s', r'sql server', r'power bi', 
    r'spark', r'postgresql', r'informatica', r'microstrategy', r'(mysql)|(mariadb)']

tech_terms_labels = [
    'Excel', 'Tableau', 'Qlik', 'Hadoop', 'SSRS/SSIS', 'SQL Server', 'Power BI', 
    'Spark', 'Postgresql', 'Informatica', 'Microstrategy', 'MySQL/MariaDB']

#### Preparing a square matrix with counts for matching filters

First we define a helper function that returns `True` for job offers where both of the provided patterns are found in the job contents. We also prepare a list of all possible filter patterns' combinations.

In [6]:
def match_terms(first_term, second_term, text):
    if re.search(first_term, text, re.IGNORECASE) and re.search(second_term, text, re.IGNORECASE):
        return True
    else:
        return False
    
def count_matches(first_term, second_term, col):
    return sum([match_terms(first_term, second_term, t) for t in col.values])

req_combinations = list(combinations(tech_terms_filters, 2))
req_combinations[:5]

[('excel', 'tableau'),
 ('excel', 'qlik'),
 ('excel', 'hadoop'),
 ('excel', 'ss[ir]s'),
 ('excel', 'sql server')]

Building the counts in a dictionary with keys composed of tuples with both search terms. 

In [7]:
%%time
match_results = [count_matches(*tup, data_df.job_contents) for tup in req_combinations]
match_dict = dict(zip(req_combinations, match_results))

CPU times: user 25.8 s, sys: 33.5 ms, total: 25.8 s
Wall time: 25.9 s


Another helper function will unpack the combinations counts into a square matrix form. Finally, a DataFrame is created from the combinations counts matrix. This is our main data source for the chord diagram. 

In [8]:
def make_matrix(headers, counts):
    res = []
    for k1 in headers:
        row = []
        for k2 in headers:
            if k1 == k2:
                row.append(0)
            else:
                curr_key = tuple([k1, k2])
                cell_value = counts.get(curr_key) if curr_key in counts else counts.get(tuple([k2, k1]))
                row.append(cell_value)
        res.append(row)
                
    return np.array(res, dtype=int)    

In [9]:
tech_terms_matrix = make_matrix(tech_terms_filters, match_dict)
tech_terms_matrix

array([[  0, 250, 237,  50, 300, 159, 136,  55,  25,  79,  23,  64],
       [250,   0, 195,  29,  81,  41,  89,  29,   5,  35,  23,  26],
       [237, 195,   0,   8,  42,  24,  89,  14,   6,  64,  17,  10],
       [ 50,  29,   8,   0,  26,  20,  14,  75,   3,   4,   3,   3],
       [300,  81,  42,  26,   0, 141,  78,  35,  11,  42,  12,  28],
       [159,  41,  24,  20, 141,   0,  64,  16,  26,  31,  10,  33],
       [136,  89,  89,  14,  78,  64,   0,  19,   0,   5,   9,   5],
       [ 55,  29,  14,  75,  35,  16,  19,   0,   5,   4,   3,  18],
       [ 25,   5,   6,   3,  11,  26,   0,   5,   0,   0,   0,  35],
       [ 79,  35,  64,   4,  42,  31,   5,   4,   0,   0,   6,   1],
       [ 23,  23,  17,   3,  12,  10,   9,   3,   0,   6,   0,   0],
       [ 64,  26,  10,   3,  28,  33,   5,  18,  35,   1,   0,   0]])

In [10]:
tech_terms_df = pd.DataFrame(tech_terms_matrix, columns=tech_terms_labels, index=tech_terms_labels)
tech_terms_df

,Excel,Tableau,Qlik,Hadoop,SSRS/SSIS,SQL Server,Power BI,Spark,Postgresql,Informatica,Microstrategy,MySQL/MariaDB
Excel,0,250,237,50,300,159,136,55,25,79,23,64
Tableau,250,0,195,29,81,41,89,29,5,35,23,26
Qlik,237,195,0,8,42,24,89,14,6,64,17,10
Hadoop,50,29,8,0,26,20,14,75,3,4,3,3
SSRS/SSIS,300,81,42,26,0,141,78,35,11,42,12,28
SQL Server,159,41,24,20,141,0,64,16,26,31,10,33
Power BI,136,89,89,14,78,64,0,19,0,5,9,5
Spark,55,29,14,75,35,16,19,0,5,4,3,18
Postgresql,25,5,6,3,11,26,0,5,0,0,0,35
Informatica,79,35,64,4,42,31,5,4,0,0,6,1


We can finally move to Plotly. 

A chord diagram encodes information in two graphical objects:

- Ideograms, represented by distinctly colored arcs of circles;
- Ribbons, that are planar shapes bounded by two quadratic Bezier curves and two arcs of circle,that can degenerate to a point;


### Ideograms Preparation

For each of our predefined tech terms we can produce a total hits count by summing up all the entries on the row (or column for that matter). That total count determines the size of each ideogram of the chart.

We are going to need a couple of helper functions to process the data in order to get ideogram ends.


In [11]:
PI = np.pi

def moduloAB(x, a, b): #maps a real number onto the unit circle identified with 
                       #the interval [a,b), b-a=2*PI
        if a>=b:
            raise ValueError('Incorrect interval ends')
        y=(x-a)%(b-a)
        return y+b if y<0 else y+a

def test_2PI(x):
    return 0<= x <2*PI

And now use them to compute the row sums and the lengths of corresponding ideograms.

In [12]:
row_sum=[np.sum(tech_terms_matrix[k,:]) for k in range(len(tech_terms_filters))]

#set the gap between two consecutive ideograms
gap=2*PI*0.005
ideogram_length=2*PI*np.asarray(row_sum)/sum(row_sum)-gap*np.ones(len(tech_terms_filters))

The next function returns the list of end angular coordinates for each ideogram arc:


In [13]:
def get_ideogram_ends(ideogram_len, gap):
    ideo_ends=[]
    left=0
    for k in range(len(ideogram_len)):
        right=left+ideogram_len[k]
        ideo_ends.append([left, right])
        left=right+gap
    return ideo_ends

ideo_ends=get_ideogram_ends(ideogram_length, gap)
ideo_ends

[[0, 1.4164485138141805],
 [1.4478644403500784, 2.2601605207816067],
 [2.2915764473175044, 3.0019546389870895],
 [3.0333705655229872, 3.248869111760869],
 [3.2802850382967668, 4.0852262195392886],
 [4.116642146075186, 4.6788716540805035],
 [4.710287580616401, 5.212627195225525],
 [5.244043121761423, 5.499468263596767],
 [5.530884190132665, 5.621349450157441],
 [5.652765376693338, 5.906089118760395],
 [5.937505045296293, 6.017463306479631],
 [6.048879233015529, 6.251769380643686]]

The function make_ideogram_arc returns equally spaced points on an ideogram arc, expressed as complex numbers in polar form:

In [14]:
def make_ideogram_arc(R, phi, a=50):
    # R is the circle radius
    # phi is the list of ends angle coordinates of an arc
    # a is a parameter that controls the number of points to be evaluated on an arc
    if not test_2PI(phi[0]) or not test_2PI(phi[1]):
        phi=[moduloAB(t, 0, 2*PI) for t in phi]
    length=(phi[1]-phi[0])% 2*PI
    nr=5 if length<=PI/4 else int(a*length/PI)

    if phi[0] < phi[1]:
        theta=np.linspace(phi[0], phi[1], nr)
    else:
        phi=[moduloAB(t, -PI, PI) for t in phi]
        theta=np.linspace(phi[0], phi[1], nr)
    return R*np.exp(1j*theta)

In [15]:
z = make_ideogram_arc(1.3, [11*PI/6, PI/17])
z

array([1.12583302-0.65j      , 1.14814501-0.60972373j,
       1.16901672-0.5686826j , 1.18842197-0.5269281j ,
       1.20633642-0.48451259j, 1.22273759-0.44148929j,
       1.23760491-0.39791217j, 1.25091973-0.3538359j ,
       1.26266534-0.30931575j, 1.27282702-0.26440759j,
       1.28139202-0.21916775j, 1.28834958-0.17365297j,
       1.29369099-0.12792036j, 1.29740954-0.08202728j,
       1.29950058-0.0360313j , 1.29996146+0.01000988j,
       1.29879163+0.0560385j , 1.29599253+0.10199682j,
       1.2915677 +0.1478272j , 1.28552267+0.19347214j,
       1.27786503+0.23887437j])

### Ribbons Preparation

The function map_data maps all matrix entries to the corresponding values in the intervals associated to ideograms:

In [16]:
def map_data(data_matrix, row_value, ideogram_length):
    mapped=np.zeros(data_matrix.shape)
    for j  in range(len(tech_terms_filters)):
        mapped[:, j]=ideogram_length*data_matrix[:,j]/row_value
    return mapped

mapped_data=map_data(tech_terms_matrix, row_sum, ideogram_length)
mapped_data


array([[0.        , 0.25697542, 0.2436127 , 0.05139508, 0.3083705 ,
        0.16343637, 0.13979463, 0.05653459, 0.02569754, 0.08120423,
        0.02364174, 0.06578571],
       [0.25289417, 0.        , 0.19725745, 0.02933572, 0.08193771,
        0.04147464, 0.09003033, 0.02933572, 0.00505788, 0.03540518,
        0.02326626, 0.02630099],
       [0.23846973, 0.19620927, 0.        , 0.00804961, 0.04226046,
        0.02414883, 0.08955193, 0.01408682, 0.00603721, 0.06439689,
        0.01710542, 0.01006201],
       [0.04585075, 0.02659344, 0.00733612, 0.        , 0.02384239,
        0.0183403 , 0.01283821, 0.06877613, 0.00275105, 0.00366806,
        0.00275105, 0.00275105],
       [0.30336979, 0.08190984, 0.04247177, 0.02629205, 0.        ,
        0.1425838 , 0.07887615, 0.03539314, 0.01112356, 0.04247177,
        0.01213479, 0.02831451],
       [0.15822034, 0.04079896, 0.02388232, 0.01990193, 0.1403086 ,
        0.        , 0.06368617, 0.01592154, 0.02587251, 0.03084799,
        0.00995096,

The array idx_sort, defined below, has on each row the indices that sort the corresponding row in mapped_data:


In [17]:
idx_sort=np.argsort(mapped_data, axis=1)
idx_sort

array([[ 0, 10,  8,  3,  7, 11,  9,  6,  5,  2,  1,  4],
       [ 1,  8, 10, 11,  3,  7,  9,  5,  4,  6,  2,  0],
       [ 2,  8,  3, 11,  7, 10,  5,  4,  9,  6,  1,  0],
       [ 3,  8, 10, 11,  9,  2,  6,  5,  4,  1,  0,  7],
       [ 4,  8, 10,  3, 11,  7,  2,  9,  6,  1,  5,  0],
       [ 5, 10,  7,  3,  2,  8,  9, 11,  1,  6,  4,  0],
       [ 6,  8,  9, 11, 10,  3,  7,  5,  4,  1,  2,  0],
       [ 7, 10,  9,  8,  2,  5, 11,  6,  1,  4,  0,  3],
       [ 6,  8,  9, 10,  3,  1,  7,  2,  4,  0,  5, 11],
       [ 8,  9, 11,  3,  7,  6, 10,  5,  1,  4,  2,  0],
       [ 8, 10, 11,  3,  7,  9,  6,  5,  4,  2,  0,  1],
       [10, 11,  9,  3,  6,  2,  7,  1,  4,  5,  8,  0]])

In [18]:
def make_ribbon_ends(mapped_data, ideo_ends,  idx_sort):
    L=mapped_data.shape[0]
    ribbon_boundary=np.zeros((L,L+1))
    for k in range(L):
        start=ideo_ends[k][0]
        ribbon_boundary[k][0]=start
        for j in range(1,L+1):
            J=idx_sort[k][j-1]
            ribbon_boundary[k][j]=start+mapped_data[k][J]
            start=ribbon_boundary[k][j]
    return [[(ribbon_boundary[k][j],ribbon_boundary[k][j+1] ) for j in range(L)] for k in range(L)]

ribbon_ends=make_ribbon_ends(mapped_data, ideo_ends,  idx_sort)
print('ribbon ends starting from the ideogram[2]\n', ribbon_ends[2])



ribbon ends starting from the ideogram[2]
 [(2.2915764473175044, 2.2915764473175044), (2.2915764473175044, 2.297613655745291), (2.297613655745291, 2.3056632669823403), (2.3056632669823403, 2.3157252810286515), (2.3157252810286515, 2.3298121006934873), (2.3298121006934873, 2.3469175245722167), (2.3469175245722167, 2.371066358283364), (2.371066358283364, 2.413326817277872), (2.413326817277872, 2.477723707174265), (2.477723707174265, 2.5672756321864365), (2.5672756321864365, 2.763484906089509), (2.763484906089509, 3.001954638987089)]


In [19]:
def control_pts(angle, radius):
    #angle is a  3-list containing angular coordinates of the control points b0, b1, b2
    #radius is the distance from b1 to the  origin O(0,0) 

    if len(angle)!=3:
        raise InvalidInputError('angle must have len =3')
    b_cplx=np.array([np.exp(1j*angle[k]) for k in range(3)])
    b_cplx[1]=radius*b_cplx[1]
    return zip(b_cplx.real, b_cplx.imag)

In [20]:
def ctrl_rib_chords(l, r, radius):
    # this function returns a 2-list containing control poligons of the two quadratic Bezier
    #curves that are opposite sides in a ribbon
    #l (r) the list of angular variables of the ribbon arc ends defining 
    #the ribbon starting (ending) arc 
    # radius is a common parameter for both control polygons
    if len(l)!=2 or len(r)!=2:
        raise ValueError('the arc ends must be elements in a list of len 2')
    return [control_pts([l[j], (l[j]+r[j])/2, r[j]], radius) for j in range(2)]

In [21]:
def make_q_bezier(b):# defines the Plotly SVG path for a quadratic Bezier curve defined by the list of its control points
    if len(b)!=3:
        raise valueError('control poligon must have 3 points')
    A, B, C=b
    return 'M '+str(A[0])+',' +str(A[1])+' '+'Q '+\
                str(B[0])+', '+str(B[1])+ ' '+\
                str(C[0])+', '+str(C[1])

b=[(1,4), (-0.5, 2.35), (3.745, 1.47)]

make_q_bezier(b)


'M 1,4 Q -0.5, 2.35 3.745, 1.47'

In [22]:
def make_ribbon_arc(theta0, theta1):

    if test_2PI(theta0) and test_2PI(theta1):
        if theta0 < theta1:
            theta0= moduloAB(theta0, -PI, PI)
            theta1= moduloAB(theta1, -PI, PI)
            if theta0*theta1>0:
                raise ValueError('incorrect angle coordinates for ribbon')

        nr=int(40*(theta0-theta1)/PI)
        if nr<=2: nr=3
        theta=np.linspace(theta0, theta1, nr)
        pts=np.exp(1j*theta)# points on arc in polar complex form

        string_arc=''
        for k in range(len(theta)):
            string_arc+='L '+str(pts.real[k])+', '+str(pts.imag[k])+' '
        return   string_arc
    else:
        raise ValueError('the angle coordinates for an arc side of a ribbon must be in [0, 2*pi]')

make_ribbon_arc(np.pi/3, np.pi/6)

'L 0.5000000000000001, 0.8660254037844386 L 0.5877852522924732, 0.8090169943749473 L 0.6691306063588582, 0.7431448254773942 L 0.7431448254773944, 0.6691306063588581 L 0.8090169943749475, 0.5877852522924731 L 0.8660254037844387, 0.49999999999999994 '

In [23]:
def make_ideo_shape(path, line_color, fill_color):
    #line_color is the color of the shape boundary
    #fill_collor is the color assigned to an ideogram
    return  dict(
                  line=dict(
                  color=line_color,
                  width=0.45
                 ),

            path=  path,
            type='path',
            fillcolor=fill_color,
            layer='below'
        )

In [99]:
temp_colors = [
'rgba(207,86,188,0.25)',
'rgba(95,194,80,0.25)',
'rgba(134,93,213,0.25)',
'rgba(157,185,53,0.25)',
'rgba(99,108,195,0.25)',
'rgba(196,173,64,0.25)',
'rgba(150,81,158,0.25)',
'rgba(67,201,134,0.25)',
'rgba(221,77,138,0.25)',
'rgba(78,141,43,0.25)',
'rgba(208,144,209,0.25)',
'rgba(80,159,95,0.25)',
'rgba(203,63,81,0.25)',
'rgba(72,197,187,0.25)',
'rgba(214,84,45,0.25)',
'rgba(77,184,223,0.25)',
'rgba(219,148,53,0.25)',
'rgba(104,139,204,0.25)',
'rgba(128,136,45,0.25)',
'rgba(156,73,105,0.25)',
'rgba(112,193,148,0.25)',
'rgba(165,86,46,0.25)',
'rgba(53,148,122,0.25)',
'rgba(220,125,130,0.25)',
'rgba(52,117,62,0.25)',
'rgba(220,151,105,0.25)',
'rgba(32,110,84,0.25)',
'rgba(143,111,47,0.25)',
'rgba(162,181,110,0.25)',
'rgba(95,108,43,0.25)',
]

l = len(tech_terms_filters)
ideo_colors=temp_colors[:l]


In [100]:
def make_ribbon(l, r, line_color, fill_color, radius=0.2):
    #l=[l[0], l[1]], r=[r[0], r[1]]  represent the opposite arcs in the ribbon 
    #line_color is the color of the shape boundary
    #fill_color is the fill color for the ribbon shape
    poligon=ctrl_rib_chords(l,r, radius)
    b,c =poligon

    return  dict(
                line=dict(
                color=line_color, width=0.5
            ),
            path =  make_q_bezier(list(b))+make_ribbon_arc(r[0], r[1])+make_q_bezier(list(c)[::-1])+make_ribbon_arc(l[1], l[0]),
            type='path',
            fillcolor=fill_color,
            layer='below'
        )

def make_self_rel(l, line_color, fill_color, radius):
    #radius is the radius of Bezier control point b_1
    b=control_pts([l[0], (l[0]+l[1])/2, l[1]], radius)
    return  dict(
                line=dict(
                color=line_color, width=0.5
            ),
            path=  make_q_bezier(b)+make_ribbon_arc(l[1], l[0]),
            type='path',
            fillcolor=fill_color,
            layer='below'
        )

def invPerm(perm):
    # function that returns the inverse of a permutation, perm
    inv = [0] * len(perm)
    for i, s in enumerate(perm):
        inv[s] = i
    return inv



In [101]:
radii_sribb=[0.4, 0.30, 0.35, 0.39, 0.12]# these value are set after a few trials 

In [119]:
shapes = []

ribbon_info=[]
for k in range(len(tech_terms_filters)):

    sigma=idx_sort[k]
    sigma_inv=invPerm(sigma)
    for j in range(k, len(tech_terms_filters)):
        if tech_terms_matrix[k][j]==0 and tech_terms_matrix[j][k]==0: continue
        eta=idx_sort[j]
        eta_inv=invPerm(eta)
        l=ribbon_ends[k][sigma_inv[j]]

        if j==k:
            shapes.append(make_self_rel(l, 'rgb(175,175,175)', ideo_colors[k], radius=radii_sribb[k]))
            z=0.9*np.exp(1j*(l[0]+l[1])/2)
            #the text below will be displayed when hovering the mouse over the ribbon
            text=tech_terms_labels[k]+' appears in '+ '{:d}'.format(tech_terms_matrix[k][k])+''+ '',
            ribbon_info.append(
                go.Scatter(
                    x=[z.real],
                    y=[z.imag],
                    mode='markers',
                    marker=dict(size=0.5, color=ideo_colors[k]),
                    text=text,
                    hoverinfo='text',),)
        else:
            r=ribbon_ends[j][eta_inv[k]]
            zi=0.9*np.exp(1j*(l[0]+l[1])/2)
            zf=0.9*np.exp(1j*(r[0]+r[1])/2)
            texti=tech_terms_labels[k]+' appears with '+tech_terms_labels[j]+' {:d}'.format(tech_terms_matrix[k][j])+' times'
            textf=tech_terms_labels[j]+' appears with '+tech_terms_labels[k]+' {:d}'.format(tech_terms_matrix[j][k])+' times'
           
            ribbon_info.append(go.Scatter(x=[zi.real],
                                       y=[zi.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color='green'),
                                       text=texti,
                                       hoverinfo='text'
                                       )
                              ),
            ribbon_info.append(go.Scatter(x=[zf.real],
                                       y=[zf.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color='blue'),
                                       text=textf,
                                       hoverinfo='text'
                                       )
                              )
            r=(r[1], r[0])#IMPORTANT!!!  Reverse these arc ends because otherwise you get
                          # a twisted ribbon
            shapes.append(make_ribbon(l, r, 'rgb(175,175,175)' , ideo_colors[k]))

In [120]:
ideograms=[]
for k in range(len(ideo_ends)):
    z= make_ideogram_arc(1.1, ideo_ends[k])
    zi=make_ideogram_arc(1.0, ideo_ends[k])
    m=len(z)
    n=len(zi)
    ideograms.append(
        go.Scatter(
            x=z.real,
            y=z.imag,
            mode='lines',
            line=dict(color=ideo_colors[k], shape='spline', width=0.25),
            text=tech_terms_labels[k]+'<br>'+'{:d}'.format(row_sum[k]),
            hoverinfo='text',),)

    path='M '
    for s in range(m):
        path+=str(z.real[s])+', '+str(z.imag[s])+' L '

    Zi=np.array(zi.tolist()[::-1])

    for s in range(m):
        path+=str(Zi.real[s])+', '+str(Zi.imag[s])+' L '
    path+=str(z.real[0])+' ,'+str(z.imag[0])

    shapes.append(make_ideo_shape(path,'rgb(150,150,150)' , ideo_colors[k]))

data = go.Data(ideograms+ribbon_info)




In [123]:
layout = go.Layout(
    paper_bgcolor=style['colors']['bg1'],            
    plot_bgcolor=style['colors']['bg1'],
    title = 'Chord Diagam Attempt',
    titlefont=style['chart_fonts']['title'],
    font = style['chart_fonts']['text'],
    height = 760,
    width = 760,
    showlegend=False,
    xaxis=dict(
        showline=False,
        zeroline=False,
        showgrid=False,
        showticklabels=False,
        title='',),
    yaxis=dict(
        showline=False,
        zeroline=False,
        showgrid=False,
        showticklabels=False,
        title='',),
    shapes=shapes,
    hovermode = 'closest',
    hoverdistance = 10,
)

In [124]:
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename = 'data_offers_tech_requirements_chord.html')

In [31]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)

### Resources:

https://plot.ly/python/filled-chord-diagram/